In [1]:
import json
import requests as rq
from bs4 import BeautifulSoup
# import pdfkit

#### Now I get that the first page of the content is hiden in this link (https://www.zhihu.com/api/v4/questions/358255792/answers) and all you need is get the question id (eg. 358255792 in this case)
#### But to get the full json content, you need to try this url (https://www.zhihu.com/api/v4/questions/63913429/answers?include=data%5B%2A%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%2Cis_recognized%2Cpaid_info%2Cpaid_info_content%3Bdata%5B%2A%5D.mark_infos%5B%2A%5D.url%3Bdata%5B%2A%5D.author.follower_count%2Cbadge%5B%2A%5D.topics%5Cu0026limit%3D5%5Cu0026offset%3D0%5Cu0026platform%3Ddesktop%5Cu0026sort_by%3Ddefault%22%2C%22totals\u0026limit=5\u0026offset=0), also in which you need to give the question's id

In [21]:
# headers are required to successfully connect to the ZhiHu sever
headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Connection' : 'Keep-Alive',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
}

In [3]:
def get_question_init_json(url):
    question_num=url.split('question/')[1].split('/')[0]
    json_link='https://www.zhihu.com/api/v4/questions/358255792/answers?include=data%5B%2A%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%2Cis_recognized%2Cpaid_info%2Cpaid_info_content%3Bdata%5B%2A%5D.mark_infos%5B%2A%5D.url%3Bdata%5B%2A%5D.author.follower_count%2Cbadge%5B%2A%5D.topics&limit=5&offset=0&platform=desktop&sort_by=default'
    return json_link.split('questions/')[0]+'questions/'+question_num+ '/' + json_link.split('questions/')[1].split('/')[1]

In [4]:
def change_header(soup):
    header='<head><meta charset="utf-8"/><link href="https://static.zhihu.com/heifetz/app.a4f856260488428f4051.css" rel="stylesheet" /></head> \n'
    header=header.encode('utf-8')
    header_tag=BeautifulSoup(header,'html.parser')
    header_tag
    soup.head.replace_with(header_tag)
    temp_text=str(soup)
    temp_text=temp_text.encode('utf-8')
    soup=BeautifulSoup(temp_text, 'html.parser')

In [5]:
def get_data(url):
    script=rq.get(url,headers=headers).text
    data = json.loads(script)
    return data

In [80]:
def add_answer(new_tag):
    global last_tag
    global soup
    last_tag.insert_after(new_tag)
    soup=BeautifulSoup(soup.prettify(), 'html.parser')
    last_tag=soup.body.find('div',{'class':'Question-main'}).find_all('div',{'class':"List-item"})[-1]

In [7]:
def insert_dividing_line(tag):
    dividing_line=BeautifulSoup('<HR style="border:3 double #01ffff" width="120%" color=#01ffff SIZE=5>', 'html.parser')
    tag.insert_after(dividing_line)

In [99]:
def build_new_tag(data):
    template='<div class="List-item" tabindex="0"><div class="AuthorInfo AnswerItem-authorInfo AnswerItem-authorInfo--related" itemprop="author" itemscope="" itemtype="http://schema.org/Person">                        <div class="AuthorInfo-content">                            <div aria-expanded="false" aria-haspopup="true" aria-owns="null-content" id="null-toggle">                                <img alt="PC小白" class="Avatar AuthorInfo-avatar" height="38" src="https://pic3.zhimg.com/v2-bf93187144b254475681a632b677ed0f_xs.jpg" srcset="https://pic3.zhimg.com/v2-bf93187144b254475681a632b677ed0f_l.jpg 2x" width="38" />                                <a class="UserLink-link" data-za-detail-view-element_name="User" href="//www.zhihu.com/people/lin-xx-35-94" target="_blank">PC小白</a>                                <div class="ztext AuthorInfo-badgeText">别发私信了，现在私信似乎没提示。</div>                            </div>                        </div>                    </div>                    <div class="RichContent RichContent--unescapable">                        <div class="RichContent-inner">                            <span class="RichText ztext CopyrightRichText-richText" itemprop="text">                            </span>                        </div>                        <div class="ContentItem-actions RichContent-actions">                            <button class="Button VoteButton VoteButton--up" type="button">                                        赞同1K                                </button>                            <button class="Button ContentItem-action Button--plain Button--withIcon Button--withLabel" type="button">                                    123条评论                                </button>                        </div>                    </div>                </div>'
    template=BeautifulSoup(template, 'html.parser')
    template=BeautifulSoup(template.prettify(), 'html.parser')
    template.a.string=data['author']['name']
    template.a['href']='//www.zhihu.com/people/' + data['author']['url_token']
    template.find('div',{'class':"ztext AuthorInfo-badgeText"}).string=data['author']['headline']
    template.img['src']=data['author']['avatar_url']
    template.find_all('button')[0].string='赞同'+ str(data['voteup_count'])
    template.find_all('button')[1].string=str(data['comment_count'])+'条评论'
    #template.span.string=data['data'][1]['content']
    str1='<span class="RichText ztext CopyrightRichText-richText" itemprop="text">'
    str2='</span>'
    content=str1+data['content']+str2
    content_tag=BeautifulSoup(content, "lxml")
    template.span.replace_with(content_tag)
    template=BeautifulSoup(template.prettify(), 'html.parser')
    return template.div

In [95]:
template='<div class="List-item" tabindex="0"><div class="AuthorInfo AnswerItem-authorInfo AnswerItem-authorInfo--related" itemprop="author" itemscope="" itemtype="http://schema.org/Person">                        <div class="AuthorInfo-content">                            <div aria-expanded="false" aria-haspopup="true" aria-owns="null-content" id="null-toggle">                                <img alt="PC小白" class="Avatar AuthorInfo-avatar" height="38" src="https://pic3.zhimg.com/v2-bf93187144b254475681a632b677ed0f_xs.jpg" srcset="https://pic3.zhimg.com/v2-bf93187144b254475681a632b677ed0f_l.jpg 2x" width="38" />                                <a class="UserLink-link" data-za-detail-view-element_name="User" href="//www.zhihu.com/people/lin-xx-35-94" target="_blank">PC小白</a>                                <div class="ztext AuthorInfo-badgeText">别发私信了，现在私信似乎没提示。</div>                            </div>                        </div>                    </div>                    <div class="RichContent RichContent--unescapable">                        <div class="RichContent-inner">                            <span class="RichText ztext CopyrightRichText-richText" itemprop="text">                            </span>                        </div>                        <div class="ContentItem-actions RichContent-actions">                            <button class="Button VoteButton VoteButton--up" type="button">                                        赞同1K                                </button>                            <button class="Button ContentItem-action Button--plain Button--withIcon Button--withLabel" type="button">                                    123条评论                                </button>                        </div>                    </div>                </div>'
template=BeautifulSoup(template, 'html.parser')
template=BeautifulSoup(template.prettify(), 'html.parser')
template.a.string=data['data'][1]['author']['name']
template.a['href']='flsaj'
template.a['href']

'flsaj'

In [98]:
data['data'][1]['author']['url_token']

'bai-yun-long-52'

In [70]:
def change_pic_source(soup):
#    original=True
#        if original:
#        else:
#            figure.find_all('img')[1]['src']=figure.find_all('img')[1]['data-actualsrc']
    for figure in soup.find_all('figure'):
        try:
            figure.find_all('img')[1]['src']=figure.find_all('img')[1]['data-original']
        except KeyError:
             figure.find_all('img')[1]['src']=figure.find_all('img')[1]['data-actualsrc']

In [22]:
def get_init_soup(url):
    requests=rq.get(url, headers=headers).text
    return BeautifulSoup(requests, 'html.parser')

In [100]:
input_url='https://www.zhihu.com/question/25416371/answer/629008593'
if 'answer' in input_url:
    input_url=input_url.split('/answer')[0]

soup=get_init_soup(input_url)

file_name = soup.title.text.replace(':','_').replace('？','?').replace(' ','').replace('，',',') + '.html'
# default path is the current path
file_path = ''
full_name = file_path+file_name
#full_name

last_tag=soup.body.find('div',{'class':'Question-main'}).find_all('div',{'class':"List-item"})[-1]

initial_url=get_question_init_json(input_url)

data=get_data(initial_url)

is_end=data['paging']['is_end']

count=0

while 1:
    if is_end==False and count!=2:
        count=count+1
        answer_url=data['paging']['next']
        data=get_data(answer_url)
        for answer_content in data['data']:
            print(answer_content['author']['name'])
            new_tag=build_new_tag(answer_content)
            add_answer(new_tag)
        is_end=data['paging']['is_end']
    else:
        for answer in soup.body.find('div',{'class':'Question-main'}).find_all('div',{'class':"List-item"}):
            insert_dividing_line(answer)
        soup=BeautifulSoup(soup.prettify(), 'html.parser')
        change_pic_source(soup)
        change_header(soup)
        print(count)
        break
        
with open(full_name, "w") as file: 
    file.write(str(soup))

Tech-IN
玩命快客
六百八十寺
知乎用户
知乎用户
苏旷
房仲闻
曾铭
知乎用户
Kenneth
2


## Single url test

In [135]:
Page=rq.get(url,headers=headers)

In [54]:
soup=BeautifulSoup(soup.prettify(), 'html.parser')

In [43]:
def change_pic_source(soup):
    original=True
    for figure in soup.find_all('figure'):
        if original:
            figure.find_all('img')[1]['src']=figure.find_all('img')[1]['data-original']
        else:
            figure.find_all('img')[1]['src']=figure.find_all('img')[1]['data-actualsrc']

<body>
<div id="root">
<div>
<div class="LoadingBar">
</div>
<div>
<header class="Sticky AppHeader" data-za-module="TopNavBar" role="banner">
<div class="AppHeader-inner">
<a aria-label="知乎" href="//www.zhihu.com">
<svg aria-hidden="true" class="Icon ZhihuLogo ZhihuLogo--blue Icon--logo" height="30" style="height:30px;width:64px" viewbox="0 0 200 91" width="64">
<title>
</title>
<g>
<path d="M53.29 80.035l7.32.002 2.41 8.24 13.128-8.24h15.477v-67.98H53.29v67.978zm7.79-60.598h22.756v53.22h-8.73l-8.718 5.473-1.587-5.46-3.72-.012v-53.22zM46.818 43.162h-16.35c.545-8.467.687-16.12.687-22.955h15.987s.615-7.05-2.68-6.97H16.807c1.09-4.1 2.46-8.332 4.1-12.708 0 0-7.523 0-10.085 6.74-1.06 2.78-4.128 13.48-9.592 24.41 1.84-.2 7.927-.37 11.512-6.94.66-1.84.785-2.08 1.605-4.54h9.02c0 3.28-.374 20.9-.526 22.95H6.51c-3.67 0-4.863 7.38-4.863 7.38H22.14C20.765 66.11 13.385 79.24 0 89.62c6.403 1.828 12.784-.29 15.937-3.094 0 0 7.182-6.53 11.12-21.64L43.92 85.18s2.473-8.402-.388-12.496c-2.37-2.788-8.768-

In [68]:
a=soup.find_all('figure')[8]
a

<figure>
<noscript>
<img class="content_image" data-rawheight="188" data-rawwidth="240" src="https://pic2.zhimg.com/50/2228a04b578efc16eb6fc135020063ff_hd.jpg" width="240"/>
</noscript>
<img class="content_image lazy" data-actualsrc="https://pic2.zhimg.com/50/2228a04b578efc16eb6fc135020063ff_hd.jpg" data-rawheight="188" data-rawwidth="240" src="data:image/svg+xml;utf8,&lt;svg xmlns='http://www.w3.org/2000/svg' width='240' height='188'&gt;&lt;/svg&gt;" width="240"/>
</figure>

In [69]:
index=0
for figure in soup.find_all('figure'):
    try:
        figure.find_all('img')[1]['src']=figure.find_all('img')[1]['data-original']
    except KeyError:
         figure.find_all('img')[1]['src']=figure.find_all('img')[1]['data-actualsrc']
    index=index+1
    print(index)

1
2
3
4
5
6
7
8
9
10
11
12
13


In [57]:
change_pic_source(soup)

KeyError: 'data-original'

In [97]:
#BlockPage.status_code
#BlockPage.headers
#BlockPage.text

In [210]:
Page_soup = BeautifulSoup(Page.text, 'html.parser')

In [36]:
title=soup.title.text
title

None


In [156]:
file_name = soup.title.text.replace(':','_').replace('？','?').replace(' ','').replace('，',',') + '.html'
# default path is the current path
file_path = ''
full_name = file_path+file_name
full_name

'世界上有哪些代码量很少,但很牛逼很经典的算法或项目案例?-知乎.html'

In [139]:
#question=Page_soup.main.div.div
#question
#question_describe=question.find_all('span',{'class':"RichText ztext"})[0]
#question_describe.text

'是不是代码越少越好？有什么典型例子吗？请教各位技术大神。'

In [10]:
#[s.extract() for s in Page_soup('script')]

#s=Page_soup.find_all('header')[0]
#s.extract()

#s=Page_soup.find('main').div.div
#s.extract()

In [165]:
#Page_soup.find_all('div',{'class':'Card AnswerCard'})

In [37]:
with open(full_name, "w") as file: 
    file.write(str(Page_soup))

In [220]:
Page_soup=BeautifulSoup(Page_soup.prettify(), 'html.parser')

In [225]:
last_item=Page_soup.body.find('div',{'class':'Question-main'}).find_all('div',{'class':"List-item"})[-1]

In [224]:
test_tag='<div class="List-item" tabindex="0"><div class="AuthorInfo AnswerItem-authorInfo AnswerItem-authorInfo--related" itemprop="author" itemscope="" itemtype="http://schema.org/Person">                        <div class="AuthorInfo-content">                            <div aria-expanded="false" aria-haspopup="true" aria-owns="null-content" id="null-toggle">                                <img alt="PC小白" class="Avatar AuthorInfo-avatar" height="38" src="https://pic3.zhimg.com/v2-bf93187144b254475681a632b677ed0f_xs.jpg" srcset="https://pic3.zhimg.com/v2-bf93187144b254475681a632b677ed0f_l.jpg 2x" width="38" />                                <a class="UserLink-link" data-za-detail-view-element_name="User" href="//www.zhihu.com/people/lin-xx-35-94" target="_blank">PC小白</a>                                <div class="ztext AuthorInfo-badgeText">别发私信了，现在私信似乎没提示。</div>                            </div>                        </div>                    </div>                    <div class="RichContent RichContent--unescapable">                        <div class="RichContent-inner">                            <span class="RichText ztext CopyrightRichText-richText" itemprop="text">                            </span>                        </div>                        <div class="ContentItem-actions RichContent-actions">                            <button class="Button VoteButton VoteButton--up" type="button">                                        赞同1K                                </button>                            <button class="Button ContentItem-action Button--plain Button--withIcon Button--withLabel" type="button">                                    123条评论                                </button>                        </div>                    </div>                </div>'
test_tag=BeautifulSoup(test_tag, 'html.parser')
test_tag=BeautifulSoup(test_tag.prettify(), 'html.parser')
test_tag

<div class="List-item" tabindex="0">
<div class="AuthorInfo AnswerItem-authorInfo AnswerItem-authorInfo--related" itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<div class="AuthorInfo-content">
<div aria-expanded="false" aria-haspopup="true" aria-owns="null-content" id="null-toggle">
<img alt="PC小白" class="Avatar AuthorInfo-avatar" height="38" src="https://pic3.zhimg.com/v2-bf93187144b254475681a632b677ed0f_xs.jpg" srcset="https://pic3.zhimg.com/v2-bf93187144b254475681a632b677ed0f_l.jpg 2x" width="38"/>
<a class="UserLink-link" data-za-detail-view-element_name="User" href="//www.zhihu.com/people/lin-xx-35-94" target="_blank">
     PC小白
    </a>
<div class="ztext AuthorInfo-badgeText">
     别发私信了，现在私信似乎没提示。
    </div>
</div>
</div>
</div>
<div class="RichContent RichContent--unescapable">
<div class="RichContent-inner">
<span class="RichText ztext CopyrightRichText-richText" itemprop="text">
</span>
</div>
<div class="ContentItem-actions RichContent-actions">
<button

In [257]:
last_item.insert_after(template.div)

In [25]:
json_link='https://www.zhihu.com/api/v4/questions/358255792/answers?include=data%5B%2A%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%2Cis_recognized%2Cpaid_info%2Cpaid_info_content%3Bdata%5B%2A%5D.mark_infos%5B%2A%5D.url%3Bdata%5B%2A%5D.author.follower_count%2Cbadge%5B%2A%5D.topics&limit=5&offset=5&platform=desktop&sort_by=default'
script=rq.get(json_link,headers=headers).text
data = json.loads(script)

In [116]:
data['data'][1]['content']

'<p>先说结论，最大的区别是产品调性。一个是工业互联网的产物，一个是还带着文艺情怀的真实产品。</p><figure data-size="normal"><noscript><img src="https://pic4.zhimg.com/50/v2-30f042bdd23e38c74c294db826d7718f_hd.jpg" data-caption="" data-size="normal" data-rawwidth="496" data-rawheight="280" class="origin_image zh-lightbox-thumb" width="496" data-original="https://pic4.zhimg.com/v2-30f042bdd23e38c74c294db826d7718f_r.jpg"/></noscript><img src="data:image/svg+xml;utf8,&lt;svg xmlns=&#39;http://www.w3.org/2000/svg&#39; width=&#39;496&#39; height=&#39;280&#39;&gt;&lt;/svg&gt;" data-caption="" data-size="normal" data-rawwidth="496" data-rawheight="280" class="origin_image zh-lightbox-thumb lazy" width="496" data-original="https://pic4.zhimg.com/v2-30f042bdd23e38c74c294db826d7718f_r.jpg" data-actualsrc="https://pic4.zhimg.com/50/v2-30f042bdd23e38c74c294db826d7718f_hd.jpg"/></figure><p>今天这篇文，不谈短视频头部交椅的抢夺，仅作为一名产品出身的创业者、一名还保存着点点情怀的产品经理，谈谈快手的一个产品细节。</p><p class="ztext-empty-paragraph"><br/></p><figure data-size="normal"><noscript><img src="ht

### Test local file

In [97]:
template = open('Zhihu_template.html', 'r', encoding='utf-8').read()
soup = BeautifulSoup(template,'html.parser')


In [99]:
temp_text=str(soup)
temp_text=temp_text.encode('utf-8')
soup=BeautifulSoup(temp_text, 'lxml')

In [101]:
with open('full_name.html', "w") as file: 
    file.write(str(soup))

In [83]:
soup.body.div.div.div.find_all('div',{'class':"List-item"})[1].span

AttributeError: can't set attribute

In [108]:
content='<span class="RichText ztext CopyrightRichText-richText" itemprop="text">'\
+data['data'][1]['content']+ '</span>'
tag=BeautifulSoup(content.encode('utf-8'), 'html.parser')
test1=soup.body.div.div.div.find_all('div',{'class':"List-item"})[1].span
test1=test1.append(tag)
soup

<!DOCTYPE doctype>
<html><body><p>tml&gt;


</p>
<link href="https://static.zhihu.com/heifetz/app.a4f856260488428f4051.css" rel="stylesheet"/>
<div class="Question-main">
<div class="Question-mainColumn">
<div class="List">
<hr color="#000000" size="5" style="border:3 double #000000" width="120%"/>
<div class="List-item" tabindex="0">
<div class="ContentItem-meta">
<h4 class="List-headerText"><span>Question's name</span></h4>
<h1 class="List-headerText"><span>Number of answer</span></h1>
<div class="RichContent-inner"><span class="RichText ztext CopyrightRichText-richText" itemprop="text">
                                Question's description                                
                        </span>
</div>
</div>
</div>
<hr color="#000000" size="5" style="border:3 double #000000" width="120%"/>
<div class="List-item" tabindex="0">
<div class="AuthorInfo AnswerItem-authorInfo AnswerItem-authorInfo--related" itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<div 